# Importando as bibliotecas necessárias

In [19]:
import numpy as np
import pandas as pd

# Lendo o arquivo de entrada, adequando para a forma correta e preparando a tabela

In [20]:
def ler_entrada(arquivo):
    try:
        with open(arquivo, 'r') as f:
            linhas = [linha.strip() for linha in f.readlines()]
        
        # Validar quantidade de linhas
        if len(linhas) < 4:
            raise ValueError("O arquivo deve ter pelo menos 4 linhas: tipo do problema, número de variáveis, número de restrições, e a função objetivo.")
        
        # Ler tipo do problema (maximização ou minimização)
        try:
            tipo_problema = int(linhas[0])
            if tipo_problema not in [-1, 1]:
                raise ValueError("O tipo do problema deve ser -1 (minimização) ou 1 (maximização).")
        except ValueError:
            raise ValueError("A primeira linha deve ser um número inteiro: -1 ou 1.")

        # Ler número de variáveis
        try:
            num_variaveis = int(linhas[1])
            if num_variaveis <= 0:
                raise ValueError("O número de variáveis deve ser um inteiro positivo.")
        except ValueError:
            raise ValueError("A segunda linha deve ser um número inteiro positivo indicando o número de variáveis.")

        # Ler número de restrições
        try:
            num_restricoes = int(linhas[2])
            if num_restricoes <= 0:
                raise ValueError("O número de restrições deve ser um inteiro positivo.")
        except ValueError:
            raise ValueError("A terceira linha deve ser um número inteiro positivo indicando o número de restrições.")

        # Validar quantidade de linhas do arquivo
        if len(linhas) != 3 + 1 + num_restricoes:
            raise ValueError(f"O arquivo deve ter exatamente {3 + 1 + num_restricoes} linhas, mas possui {len(linhas)}.")
        
        # Ler a função objetivo
        try:
            func_objetivo = [float(x) for x in linhas[3].replace(',', ' ').split()]
            if len(func_objetivo) != num_variaveis:
                raise ValueError("O número de coeficientes da função objetivo não corresponde ao número de variáveis.")
        except ValueError:
            raise ValueError("A quarta linha deve conter os coeficientes da função objetivo, separados por vírgulas ou espaços.")
        
        # Ler as restrições
        restricoes = []
        for i in range(num_restricoes):
            try:
                restricao = [float(x) for x in linhas[4 + i].replace(',', ' ').split()]
                if len(restricao) != num_variaveis + 1:
                    raise ValueError(f"A restrição na linha {5 + i} não possui o número correto de coeficientes (variáveis + lado direito).")
                restricoes.append(restricao)
            except ValueError:
                raise ValueError(f"A linha {5 + i} (restrição) está mal formatada. Deve conter {num_variaveis + 1} valores numéricos separados por vírgulas ou espaços.")
        
        return tipo_problema, num_variaveis, num_restricoes, func_objetivo, restricoes

    except FileNotFoundError:
        raise FileNotFoundError(f"O arquivo '{arquivo}' não foi encontrado.")
    except Exception as e:
        raise RuntimeError(f"Erro ao ler o arquivo: {e}")


In [21]:
def preparar_tabela(tipo_problema, num_variaveis, num_restricoes, func_objetivo, restricoes):
    # Se for minimização, transformar coeficientes da função objetivo em negativos
    if tipo_problema == -1:
        func_objetivo = [-c for c in func_objetivo]
    
    # Adicionar coeficientes das variáveis de folga
    tabela = []
    for i in range(num_restricoes):
        linha = restricoes[i][:num_variaveis] + [0] * num_restricoes + [restricoes[i][-1]]
        linha[num_variaveis + i] = 1  # Adiciona 1 na coluna da variável de folga
        tabela.append(linha)
    
    # Adicionar a função objetivo à tabela na primeira linha
    linha_objetivo = func_objetivo + [0] * (num_restricoes + 1)
    tabela.insert(0, [-c for c in linha_objetivo])  # Negar para maximização
    
    return np.array(tabela, dtype=float)


In [22]:
# Teste de leitura e preparação da tabela
arquivo = 'Entrada.txt'
tipo_problema, num_variaveis, num_restricoes, func_objetivo, restricoes = ler_entrada(arquivo)
tabela = preparar_tabela(tipo_problema, num_variaveis, num_restricoes, func_objetivo, restricoes)

print("Tabela inicial:")
print(tabela)

Tabela inicial:
[[-3. -5.  0.  0.  0.  0.]
 [ 2.  1.  1.  0.  0. 30.]
 [ 3. -1.  0.  1.  0. 20.]
 [ 1.  4.  0.  0.  1. 15.]]


# Implementando o Simplex

In [23]:
def ajusta_tabela(tabela, linha_pivo, coluna_pivo):
    # Normaliza a linha pivô, fazendo a divisão da linha pelo elemento pivô
    tabela[linha_pivo] /= tabela[linha_pivo, coluna_pivo]

    # Atualiza as outras linhas para zerar o coeficiente da coluna pivô.
    for linha in range(tabela.shape[0]):
        if linha != linha_pivo:
            # Nova linha = linha atual - coeficiente da linha na coluna pivô * coeficiente da coluna na linha pivô
            tabela[linha] -= tabela[linha, coluna_pivo] * tabela[linha_pivo]  

    return tabela

def simplex(tabela):
    num_linhas, num_colunas = tabela.shape

    while True:
        # Se todos os coeficientes na linha da função objetivo são positivos, a solução ótima foi encontrada.
        if all(tabela[0, :-1] >= 0):
            break

        # Seleciona a coluna com o coeficiente mais negativo na linha da função objetivo. (Variável que entrará na base)
        coluna_pivo = np.argmin(tabela[0, :-1])

        # Calcula as razões entre o lado direito (LD) e os coeficientes da coluna pivô.
        # A linha com a menor razão positiva é selecionada. (Variável que sairá da base)
        razoes = []
        for linha in range(1, num_linhas):  # Ignora a linha da função objetivo
            if tabela[linha, coluna_pivo] > 0:
                razoes.append(tabela[linha, -1] / tabela[linha, coluna_pivo])  # Razão = LD / Coeficiente da coluna pivô
            else:
                razoes.append(float('inf'))  # Ignora coeficientes negativos ou zero

        linha_pivo = np.argmin(razoes) + 1  # +1 para ajustar o índice (a primeira linha é a função objetivo)

        # Método externo para ajustar a tabela.
        tabela = ajusta_tabela(tabela, linha_pivo, coluna_pivo)


    # Extração da Solução Ótima:
    solucao = np.zeros(num_colunas - 1)  # Vetor para armazenar os valores das variáveis (esse np.zeros inicia um vetor com zeros) (tamanho da quantidade de colunas incluindo variáveis de folga)
    for linha in range(1, num_linhas):
        # Verifica se a variável é básica (coluna com exatamente um 1 e o resto 0).
        coluna_basica = np.where(tabela[linha, :-1] == 1)[0]
        # Primeiro, verifica se há apenas uma coluna com valor 1 na linha atual e depois verifica se a coluna segue o formato de variável básica (tudo zero e apenas um valor 1)
        if len(coluna_basica) == 1 and all(tabela[:, coluna_basica[0]] == [0] * linha + [1] + [0] * (num_linhas - linha - 1)):
            solucao[coluna_basica[0]] = tabela[linha, -1]  # Armazena o valor da variável básica (valor que ela tem no LD) no vetor solução

    valor_otimo = tabela[0, -1]

    return solucao, valor_otimo

# Melhorando a visualização

In [31]:
def exibir_tabela_com_pandas(tabela, num_variaveis, num_restricoes):
    # Criando rótulos das colunas
    colunas = [f"x{i+1}" for i in range(num_variaveis)] + [f"s{i+1}" for i in range(num_restricoes)] + ["LD"]
    # Criando rótulos das linhas
    linhas = ["Z"] + [f"Restrição {i+1}" for i in range(num_restricoes)]
    # Transformando tabela em DataFrame
    df = pd.DataFrame(tabela, columns=colunas, index=linhas)
    return df

def exibir_solucao_com_pandas(solucao, valor_otimo):
    variaveis = [f"x{i+1}" for i in range(num_variaveis)] + [f"s{i+1}" for i in range(num_restricoes)]
    df_solucao = pd.DataFrame({"Variável": variaveis, "Valor": solucao})
    df_solucao.loc[len(df_solucao)] = ["Z (Ótimo)", valor_otimo]
    return df_solucao


# Testando o código

In [32]:
arquivo = 'Entrada.txt'
tipo_problema, num_variaveis, num_restricoes, func_objetivo, restricoes = ler_entrada(arquivo)
tabela = preparar_tabela(tipo_problema, num_variaveis, num_restricoes, func_objetivo, restricoes)

print("Tabela inicial:")
exibir_tabela_com_pandas(tabela, num_variaveis, num_restricoes)

Tabela inicial:


,x1,x2,s1,s2,s3,LD
Z,-3.0,-5.0,0.0,0.0,0.0,0.0
Restrição 1,2.0,1.0,1.0,0.0,0.0,30.0
Restrição 2,3.0,-1.0,0.0,1.0,0.0,20.0
Restrição 3,1.0,4.0,0.0,0.0,1.0,15.0


In [33]:
solucao, valor_otimo = simplex(tabela)

df_solucao = exibir_solucao_com_pandas(solucao, valor_otimo)

print("\nSolução ótima:")
df_solucao


Solução ótima:


,Variável,Valor
0,x1,7.307692
1,x2,1.923077
2,s1,13.461538
3,s2,0.000000
4,s3,0.000000
5,Z (Ótimo),31.538462
